# setup

In [81]:
from optibook.synchronous_client import Exchange

import logging
logger = logging.getLogger('client')
logger.setLevel('ERROR')

print("Setup was successful.")

Setup was successful.


In [136]:
e = Exchange()
a = e.connect()

2021-03-06 22:42:32,826 [asyncio   ] [MainThread  ] Using selector: EpollSelector
2021-03-06 22:42:32,971 [client    ] [Thread-7    ] Forcing a disconnect due to an error: Closing connection because someone else logged in with the same credentials. Only one session may be active at the same time.


In [5]:
e.disconnect()









# finding the delta

In [137]:
# just for ask
amazon_book = e.get_last_price_book("AMAZON")
google_book = e.get_last_price_book("GOOGLE")
basket_book = e.get_last_price_book("TECH_BASKET")

In [8]:
min(amazon_book.asks, key=(lambda x: x.price)), min(google_book.asks, key=(lambda x: x.price))

([price_volume] price=116.30000000000001, volume=246,
 [price_volume] price=91.30000000000001, volume=248)

In [9]:
basket_book.asks

[[price_volume] price=208.60000000000002, volume=25,
 [price_volume] price=208.8, volume=25,
 [price_volume] price=208.9, volume=50]

In [10]:
121.1 + 85.2 - 206.7

-0.39999999999997726

In [11]:
#theoretical to actual
def delta_ask():
    amazon_book = e.get_last_price_book("AMAZON").asks
    google_book = e.get_last_price_book("GOOGLE").asks
    basket_book = e.get_last_price_book("TECH_BASKET").asks
    
    while len(amazon_book) == 0 or len(google_book) == 0 or len(basket_book) == 0:
        amazon_book = e.get_last_price_book("AMAZON").asks
        google_book = e.get_last_price_book("GOOGLE").asks
        basket_book = e.get_last_price_book("TECH_BASKET").asks
        
    
    theoretical_ask = (min(amazon_book, key=(lambda x: x.price)), min(google_book, key=(lambda x: x.price)))
    actual_ask = min(basket_book, key=(lambda x: x.price))
    
    return (theoretical_ask[0].price + theoretical_ask[1].price - actual_ask.price, min(theoretical_ask[0].volume, theoretical_ask[1].volume, actual_ask.volume))

In [12]:
delta_ask()

(-0.6999999999999886, 25)

In [13]:
def execute_ask(max_volume=250):
    amazon_book = e.get_last_price_book("AMAZON").asks
    google_book = e.get_last_price_book("GOOGLE").asks
    basket_book = e.get_last_price_book("TECH_BASKET").asks
    
    while len(amazon_book) == 0 or len(google_book) == 0 or len(basket_book) == 0:
        amazon_book = e.get_last_price_book("AMAZON").asks
        google_book = e.get_last_price_book("GOOGLE").asks
        basket_book = e.get_last_price_book("TECH_BASKET").asks
    
    amazon_ask = min(amazon_book, key=(lambda x: x.price))
    google_ask = min(google_book, key=(lambda x: x.price))
    basket_ask = min(basket_book, key=(lambda x: x.price))
    
    volume = min(max_volume, amazon_ask.volume, google_ask.volume, basket_ask.volume)
    
    if amazon_ask.price + google_ask.price > basket_ask.price:
        e.insert_order("AMAZON", price=amazon_ask.price, volume=volume, side="ask", order_type="limit")
        e.insert_order("GOOGLE", price=google_ask.price, volume=volume, side="ask", order_type="limit")
        
        e.insert_order("TECH_BASKET", price=basket_ask.price, volume=volume, side="bid", order_type="limit")
    elif amazon_ask.price + google_ask.price < basket_ask.price:
        e.insert_order("TECH_BASKET", price=basket_ask.price, volume=volume, side="ask", order_type="limit")
        
        e.insert_order("AMAZON", price=amazon_ask.price, volume=volume, side="bid", order_type="limit")
        e.insert_order("GOOGLE", price=google_ask.price, volume=volume, side="bid", order_type="limit")

In [24]:
def execute_bid(max_volume=250):
    margin = 0.9
    amazon_book = e.get_last_price_book("AMAZON").bids
    google_book = e.get_last_price_book("GOOGLE").bids
    basket_book = e.get_last_price_book("TECH_BASKET").bids
    
    while len(amazon_book) == 0 or len(google_book) == 0 or len(basket_book) == 0:
        amazon_book = e.get_last_price_book("AMAZON").bids
        google_book = e.get_last_price_book("GOOGLE").bids
        basket_book = e.get_last_price_book("TECH_BASKET").bids
    
    amazon_bid = min(amazon_book, key=(lambda x: x.price))
    google_bid = min(google_book, key=(lambda x: x.price))
    basket_bid = min(basket_book, key=(lambda x: x.price))
    
    volume = min(max_volume, amazon_bid.volume, google_bid.volume, basket_bid.volume)
    
    delta = amazon_bid.price + google_bid.price - basket_bid.price
    if delta > 0:
        print(amazon_bid.price + google_bid.price, basket_bid.price, amazon_bid.price - (1-margin) * delta, google_bid.price - (1-margin) * delta)
        return
        e.insert_order("AMAZON", price=amazon_bid.price - , volume=volume, side="ask", order_type="limit")
        e.insert_order("GOOGLE", price=google_bid.price, volume=volume, side="ask", order_type="limit")
        
        e.insert_order("TECH_BASKET", price=basket_bid.price, volume=volume, side="bid", order_type="limit")
    elif amazon_bid.price + google_bid.price < basket_bid.price:
        print(amazon_bid.price + google_bid.price, basket_bid.price, basket_bid.price - (1-margin) * delta)

        return
        e.insert_order("TECH_BASKET", price=basket_bid.price, volume=volume, side="ask", order_type="limit")
        
        e.insert_order("AMAZON", price=amazon_bid.price, volume=volume, side="bid", order_type="limit")
        e.insert_order("GOOGLE", price=google_bid.price, volume=volume, side="bid", order_type="limit")

In [25]:
execute_bid(100)

206.10000000000002 205.60000000000002 118.95 87.05000000000001


In [84]:
instruments = ["AMAZON", "GOOGLE", "TECH_BASKET"]

In [117]:
for instrument in instruments:
    outstanding = e.get_outstanding_orders(instrument)
    for key, status in e.get_outstanding_orders(instrument).items():
        print(f"{instrument}: {status.side} {status.volume}@{status.price}")

In [118]:
e.get_positions_and_cash()

{'AMAZON': {'volume': 0, 'cash': 366.1000000000058},
 'GOOGLE': {'volume': 0, 'cash': -1054.1999999999916},
 'TECH_BASKET': {'volume': -347, 'cash': 71228.9}}

In [87]:
def bail():
    # Get out of all positions you are currently holding, regarless of the loss involved. That means selling whatever
    # you are long, and buying-back whatever you are short. Be sure you know what you are doing when you use this logic.
    print(e.get_positions())
    for s, p in e.get_positions().items():
        if p > 0:
            e.insert_order(s, price=1, volume=p, side='ask', order_type='ioc')
        elif p < 0:
            e.insert_order(s, price=100000, volume=-p, side='bid', order_type='ioc')  
    print(e.get_positions())

In [138]:
def execute(side, max_volume, margin = 0.9):
    instruments = ["AMAZON", "GOOGLE", "TECH_BASKET"]

    def get_available_price(instrument, side):
        book = e.get_last_price_book(instrument)
        if side == "bid":
            return max(book.bids, key=(lambda x: x.price)) if len(book.bids) != 0 else None
        elif side == "ask":
            return min(book.asks, key=(lambda x: x.price)) if len(book.asks) != 0 else None
        
    def get_prices(side):
        prices = { instrument: get_available_price(instrument, side) for instrument in instruments }
        
        while None in prices.values():
            prices = { instrument: get_available_price(instrument, side) for instrument in instruments }
            
        return prices
    
    
    prices = get_prices(side)
    amazon_bid = prices.get("AMAZON")
    google_bid = prices.get("GOOGLE")
    basket_bid = prices.get("TECH_BASKET")
    
    volume = min(max_volume, amazon_bid.volume, google_bid.volume, basket_bid.volume)
    
    delta = amazon_bid.price + google_bid.price - basket_bid.price
    if delta > 0:
        if side == 'ask':
            #print(amazon_bid.price + google_bid.price, basket_bid.price, amazon_bid.price - (1-margin) * delta, google_bid.price - (1-margin) * delta)
            
            e.insert_order("AMAZON", price=amazon_bid.price - 0.1, volume=volume, side="ask", order_type="limit")
            e.insert_order("GOOGLE", price=google_bid.price - 0.1, volume=volume, side="ask", order_type="limit")

            e.insert_order("TECH_BASKET", price=basket_bid.price, volume=volume, side="bid", order_type="ioc")
            
        elif side == 'bid':
            #print(amazon_bid.price + google_bid.price, basket_bid.price, amazon_bid.price - (1-margin) * delta, google_bid.price - (1-margin) * delta)
            
            e.insert_order("AMAZON", price=amazon_bid.price, volume=volume, side="ask", order_type="ioc")
            e.insert_order("GOOGLE", price=google_bid.price, volume=volume, side="ask", order_type="ioc")

            e.insert_order("TECH_BASKET", price=basket_bid.price + 0.1, volume=volume, side="bid", order_type="limit")
    elif amazon_bid.price + google_bid.price < basket_bid.price:
        if side == 'ask':
            #print(amazon_bid.price + google_bid.price, basket_bid.price, basket_bid.price - (1-margin) * delta)

                                                        # plus margin cuz delta is already negative
            e.insert_order("TECH_BASKET", price=basket_bid.price + 0.1, volume=volume, side="ask", order_type="limit")

            e.insert_order("AMAZON", price=amazon_bid.price, volume=volume, side="bid", order_type="ioc")
            e.insert_order("GOOGLE", price=google_bid.price, volume=volume, side="bid", order_type="ioc")
        elif side == 'bid':
            #print(amazon_bid.price + google_bid.price, basket_bid.price, basket_bid.price - (1-margin) * delta)

                                                        # plus margin cuz delta is already negative
            e.insert_order("TECH_BASKET", price=basket_bid.price, volume=volume, side="ask", order_type="ioc")

            e.insert_order("AMAZON", price=amazon_bid.price - 0.1, volume=volume, side="bid", order_type="limit")
            e.insert_order("GOOGLE", price=google_bid.price - 0.1, volume=volume, side="bid", order_type="limit")

In [102]:
execute("bid", 100)

212.3 211.10000000000002 113.98 98.08


In [90]:
e.get_positions_and_cash()

{'AMAZON': {'volume': -300, 'cash': 34417.20000000001},
 'GOOGLE': {'volume': -300, 'cash': 28188.30000000001},
 'TECH_BASKET': {'volume': 48, 'cash': -11192.800000000001}}

In [91]:
from time import sleep

In [100]:
def hit_and_quit():
    abort = -3000
    vol = 250
    margin = 0
    interval = 10
    flip_flop = False
    while e.get_pnl() > abort:
        execute("bid" if flip_flop else "ask", vol, margin=margin)
        #flip_flop = not flip_flop
        
        #profit comes from the hedge, so don't bother holding stocks
        sleep(interval)
        bail()
        sleep(interval)
        

In [139]:
hit_and_quit()

KeyboardInterrupt: 

In [140]:
bail()

{'AMAZON': -98, 'GOOGLE': -98, 'TECH_BASKET': -281}
{'AMAZON': 0, 'GOOGLE': 0, 'TECH_BASKET': -206}


In [ ]:
e.get_pnl()

In [141]:
def double_down():
    vol = 250
    margin = 0
    interval = 10
    while True:
        execute("bid", vol, margin=margin)
        sleep(interval)
        execute("ask", vol, margin=margin)
        sleep(interval)

In [142]:
double_down()

2021-03-06 22:49:07,974 [client    ] [Thread-8    ] Forcing a disconnect due to an error: Pre-trade position limit (-500) breached on instrument AMAZON. Old position=-471 New position=-712.


KjException: (remote):0: failed: remote exception: std::exception: Pre-trade position limit (-500) breached on instrument AMAZON. Old position=-471 New position=-712
stack: 7fec70c21e80 7fec70c207b0 7fec70ba7666 7fec70b96572 7fec70b966a9 7fec70b8a6a4 7fec70ae6069 7fec70aed998 7fec86f80a9a 7fec86eefe7e 7fec79a70cba 7fec79a72293 7fec86edf152 7fec86f7cdb8 7fec86f86369 7fec86f86792 7fec86f7b146 7fec86f86369 7fec86f86792 7fec86f7b146 7fec86f86369 7fec86f86792 7fec86f7b146 7fec86f86369 7fec86f86792 7fec86f7b146 7fec86f878c1 7fec86edf3cd 7fec86edf4d1 7fec86edf152 7fec86f7cdb8 7fec86f86369

In [129]:
e.insert_order("TECH_BASKET", price=300, volume=20, side="ask", order_type="limit")

567389

In [130]:
instruments = ["TECH_BASKET","AMAZON","GOOGLE"]
for instrument in instruments:
    orders = e.get_outstanding_orders(instrument)
    print(orders)

{567389: <optibook.common_types.OrderStatus object at 0x7fec50fef518>}
{}
{}


In [131]:
#########################################

In [135]:
def get_available_price(instrument, side):
    book = e.get_last_price_book(instrument)
    if side == "bid":
        return max(book.bids, key=(lambda x: x.price)) if len(book.bids) != 0 else None
    elif side == "ask":
        return min(book.asks, key=(lambda x: x.price)) if len(book.asks) != 0 else None
        
def get_prices(side, instruments=["AMAZON", "GOOGLE", "TECH_BASKET"]):
    prices = { instrument: get_available_price(instrument, side) for instrument in instruments }
        
    while None in prices.values():
        prices = { instrument: get_available_price(instrument, side) for instrument in instruments }
            
    return prices

def get_single_prices(side):
    prices = { instrument: get_available_price(instrument, side) for instrument in instruments }
        
    while None in prices.values():
        prices = { instrument: get_available_price(instrument, side) for instrument in instruments }
            
    return prices
    
    
prices = get_prices("ask")
amazon_bid = prices.get("AMAZON")
google_bid = prices.get("GOOGLE")
basket_bid = prices.get("TECH_BASKET")

In [ ]:
def execute2(side, name_stock):
    instruments = ["TECH_BASKET","AMAZON","GOOGLE"]
    
    for instrument in instruments:
        orders = e.get_outstanding_orders(instrument)
        
        for order in orders:    
            if book_amazon.price+book_google<=order.price:
                if orders.instrument_id == "TECH_BASKET":
                    volume = orders.volume

                    # how much is available on the other side
                    book_amazon = get_available_price("AMAZON", side)
                    book_google = get_available_price("GOOGLE", side)
            
            
                    
            
        
    
    
    

In [143]:
instruments = ["TECH_BASKET","AMAZON","GOOGLE"]

orders = e.get_outstanding_orders("TECH_BASKET")
for order in orders.values():
    

SyntaxError: unexpected EOF while parsing (<ipython-input-143-6874a663e7e8>, line 5)

In [ ]:
def set_limit(volume):
    prices = get_prices()
    
    amazon = prices.get("AMAZON")
    google = prices.get("GOOGLE")
    basket = prices.get("TECH_BASKET")
    
    if amazon.price + google.price < basket.price:
        e.insert_order("TECH_BASKET", price = basket.price, volume=volume, side="ask")

In [ ]:
def fill_limit():
    prices = get_prices("ask", instruments=["AMAZON", "GOOGLE"])
    for trade in e.poll_new_trades("TECH_BASKET"):
        
    